In [2]:
pip install tensorflow

     ---------------------------------------- 0.0/284.2 MB ? eta -:--:--
     --------------------------------------- 0.5/284.2 MB 15.5 MB/s eta 0:00:19
     --------------------------------------- 1.0/284.2 MB 12.3 MB/s eta 0:00:23
     ---------------------------------------- 1.2/284.2 MB 9.9 MB/s eta 0:00:29
     ---------------------------------------- 1.6/284.2 MB 9.0 MB/s eta 0:00:32
     ---------------------------------------- 1.8/284.2 MB 8.0 MB/s eta 0:00:36
     ---------------------------------------- 2.1/284.2 MB 8.0 MB/s eta 0:00:36
     ---------------------------------------- 2.5/284.2 MB 8.1 MB/s eta 0:00:35
     ---------------------------------------- 2.9/284.2 MB 8.1 MB/s eta 0:00:35
     ---------------------------------------- 3.5/284.2 MB 8.5 MB/s eta 0:00:33
      --------------------------------------- 3.6/284.2 MB 8.0 MB/s eta 0:00:36
      --------------------------------------- 3.8/284.2 MB 7.5 MB/s eta 0:00:38
      --------------------------------------- 4

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import time
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras import layers

In [4]:
data = pd.read_csv('emotions.csv')

In [5]:
X = data.drop('label', axis=1)
y = data['label']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [8]:
X_train_cnn = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_cnn = X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1)

X_train_rnn = X_train.values.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_rnn = X_test.values.reshape(X_test.shape[0], X_test.shape[1], 1)

In [9]:
cnn_model = keras.Sequential([
    layers.Input(shape=(X_train_cnn.shape[1], X_train_cnn.shape[2])),  # Input layer
    layers.Conv1D(64, 3, activation='relu'),  # Convolutional layer
    layers.MaxPooling1D(2),  # Max-pooling layer
    layers.Flatten(),  # Flatten the output
    layers.Dense(128, activation='relu'),  # Fully connected layer
    layers.Dense(y_train_encoded.max() + 1, activation='softmax')  # Output layer
])

In [10]:
cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [11]:
cnn_start_time = time.time()
cnn_model.fit(X_train_cnn, y_train_encoded, epochs=10, batch_size=32, validation_split=0.2)
cnn_end_time = time.time()
cnn_time_cost = cnn_end_time - cnn_start_time

Epoch 1/10
43/43 [==============================] - 6s 129ms/step - loss: 133493514305536.0000 - accuracy: 0.4377 - val_loss: 41333246918656.0000 - val_accuracy: 0.5191
Epoch 2/10
43/43 [==============================] - 5s 123ms/step - loss: 10787383934976.0000 - accuracy: 0.5176 - val_loss: 2344754872320.0000 - val_accuracy: 0.5249
Epoch 3/10
43/43 [==============================] - 5s 126ms/step - loss: 2110234296320.0000 - accuracy: 0.5213 - val_loss: 960398491648.0000 - val_accuracy: 0.5220
Epoch 4/10
43/43 [==============================] - 5s 123ms/step - loss: 380105359360.0000 - accuracy: 0.5227 - val_loss: 651623989248.0000 - val_accuracy: 0.5279
Epoch 5/10
43/43 [==============================] - 5s 123ms/step - loss: 236519538688.0000 - accuracy: 0.5359 - val_loss: 305700077568.0000 - val_accuracy: 0.5249
Epoch 6/10
43/43 [==============================] - 5s 122ms/step - loss: 251864612864.0000 - accuracy: 0.5007 - val_loss: 321904934912.0000 - val_accuracy: 0.4692
Epoch 7

In [12]:
cnn_loss, cnn_accuracy = cnn_model.evaluate(X_test_cnn, y_test_encoded)
print("CNN Accuracy:", cnn_accuracy)
print("CNN Time Cost (s):", cnn_time_cost)

14/14 [==============================] - 0s 18ms/step - loss: 53745242112.0000 - accuracy: 0.2155
CNN Accuracy: 0.21545667946338654
CNN Time Cost (s): 53.90382242202759


In [13]:
rnn_model = keras.Sequential([
    layers.Input(shape=(X_train_rnn.shape[1], X_train_rnn.shape[2])),  # Input layer for time-series data
    layers.LSTM(64, return_sequences=True),  # LSTM layer
    layers.LSTM(64),  # LSTM layer
    layers.Dense(y_train_encoded.max() + 1, activation='softmax')  # Output layer
])

In [14]:
rnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [15]:
rnn_start_time = time.time()
rnn_model.fit(X_train_rnn, y_train_encoded, epochs=10, batch_size=32, validation_split=0.2)
rnn_end_time = time.time()
rnn_time_cost = rnn_end_time - rnn_start_time

Epoch 1/10
43/43 [==============================] - 116s 3s/step - loss: 0.5911 - accuracy: 0.7544 - val_loss: 0.3182 - val_accuracy: 0.8886
Epoch 2/10
43/43 [==============================] - 119s 3s/step - loss: 0.3340 - accuracy: 0.8768 - val_loss: 0.3054 - val_accuracy: 0.9062
Epoch 3/10
43/43 [==============================] - 130s 3s/step - loss: 0.3449 - accuracy: 0.8651 - val_loss: 0.2997 - val_accuracy: 0.8856
Epoch 4/10
43/43 [==============================] - 138s 3s/step - loss: 0.3145 - accuracy: 0.8717 - val_loss: 0.3043 - val_accuracy: 0.8886
Epoch 5/10
43/43 [==============================] - 148s 3s/step - loss: 0.2974 - accuracy: 0.8768 - val_loss: 0.5789 - val_accuracy: 0.8446
Epoch 6/10
43/43 [==============================] - 149s 3s/step - loss: 0.4295 - accuracy: 0.8490 - val_loss: 0.2963 - val_accuracy: 0.9032
Epoch 7/10
43/43 [==============================] - 155s 4s/step - loss: 0.3186 - accuracy: 0.8805 - val_loss: 0.3006 - val_accuracy: 0.8974
Epoch 8/10
43

In [16]:
rnn_loss, rnn_accuracy = rnn_model.evaluate(X_test_rnn, y_test_encoded)
print("RNN Accuracy:", rnn_accuracy)
print("RNN Time Cost (s):", rnn_time_cost)

14/14 [==============================] - 9s 620ms/step - loss: 0.2622 - accuracy: 0.8993
RNN Accuracy: 0.8992974162101746
RNN Time Cost (s): 1431.6438248157501
